# [MIRACUM 2019][Session 4] Solution
* Kim Hee (Graduate research assistant) 
* Universitätsmedizin Mannheim, Mannheim (UMM)
* This is prepared for a turorial `Data analysis tools (Datenanalysewerkzeuge)`

### Protocol:
<b>1. Create a Spark Context</b><br>
</b>2. Import required libraries</b><br>
</b>3. Load the cholera death data</b><br>
</b>4. Create a KMeans model</b><br>
</b>5. Assign centroids to the data</b><br>
</b>6. Visualize the result</b><br>

In [1]:
from pyspark import SparkContext
sc = SparkContext()
sc

<SparkContext master=local[*] appName=pyspark-shell>

### Protocol:
</b>1. Create a Spark Context</b><br>
<b>2. Import required libraries</b><br>
</b>3. Load the cholera death data</b><br>
</b>4. Create a KMeans model</b><br>
</b>5. Assign centroids to the data</b><br>
</b>6. Visualize the result</b><br>

In [2]:
import pyspark.mllib.clustering
from pyspark.mllib.clustering import KMeans, KMeansModel
import pandas as pd
import numpy as np
import folium
from folium import plugins
import matplotlib.pyplot as plt
%matplotlib inline

### Protocol:
</b>1. Create a Spark Context</b><br>
</b>2. Import required libraries</b><br>
<b>3. Load the cholera death data</b><br>
</b>4. Create a KMeans model</b><br>
</b>5. Assign centroids to the data</b><br>
</b>6. Visualize the result</b><br>

In [3]:
raw = sc.textFile("data/cholera_deaths.csv")
header = sc.parallelize([raw.first()])
raw = raw.subtract(header)
raw.take(2)

['1,2,-0.137883,51.513361', '13,1,-0.138378,51.513169']

In [4]:
line = raw.map(lambda x: x.split(','))
line.take(2)

[['1', '2', '-0.137883', '51.513361'], ['13', '1', '-0.138378', '51.513169']]

In [5]:
geo = line.map(lambda x: (float(x[2]), float(x[3])))
geo.take(2)

[(-0.137883, 51.513361), (-0.138378, 51.513169)]

### Protocol:
</b>1. Create a Spark Context</b><br>
</b>2. Import required libraries</b><br>
</b>3. Load the cholera death data</b><br>
<b>4. Create a KMeans model with `k=3`</b><br>
</b>5. Assign centroids to the data</b><br>
</b>6. Visualize the result</b><br>

In [6]:
k = 3
model = KMeans.train(geo, k)

### Protocol:
</b>1. Create a Spark Context</b><br>
</b>2. Import required libraries</b><br>
</b>3. Load the cholera death data</b><br>
</b>4. Create a KMeans model</b><br>
<b>5. Assign centroids to the data</b><br>
</b>6. Visualize the result</b><br>

In [7]:
clusters = model.predict(geo)
clusters.take(20)

[2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]

### Protocol:
</b>1. Create a Spark Context</b><br>
</b>2. Import required libraries</b><br>
</b>3. Load the cholera death data</b><br>
</b>4. Create a KMeans model</b><br>
</b>5. Assign centroids to the data</b><br>
<b>6. Visualize the result</b><br>

In [8]:
# data preparatoin for VIZ
lat = geo.map(lambda x: float(x[0])).collect()
lon = geo.map(lambda x: float(x[1])).collect()
clusters = clusters.collect()

df_list = list(map(lambda x,y,z:[x,y,z], lon,lat,clusters))
df_list[:5]

[[51.513361, -0.137883, 2],
 [51.513169, -0.138378, 2],
 [51.513402, -0.139045, 2],
 [51.513676, -0.138887, 2],
 [51.512885, -0.138624, 2]]

In [9]:
# the latitude and Longitude coordinates of the Soho District center
SOHO_COORDINATES = (51.513578, -0.136722)
map_soho = folium.Map(location = SOHO_COORDINATES, width = "100%", zoom_start = 15) # max zoom: 18
folium.TileLayer('cartodbpositron').add_to(map_soho) # stamentoner
map_soho

In [10]:
def my_palettes(cluster):
    if cluster ==  0:
        return 'green'
    elif cluster ==  1:
        return 'yellow'
    elif cluster ==  2:
        return 'red'
    elif cluster ==  3:
        return 'blue'
    elif cluster ==  4:
        return 'orange'
    elif cluster ==  5:
        return 'skyblue'
    elif cluster ==  6:
        return 'gray'
    elif cluster ==  7:
        return 'purple'
    else:
        return 'black'

In [11]:
for i in range(0, len(df_list)):
    folium.RegularPolygonMarker(location = df_list[i][:2], \
                                stroke = False, \
                                fill_color = my_palettes(df_list[i][2]), \
                                fill_opacity = 0.5, number_of_sides = 12,  radius = 3
                               ).add_to(map_soho) 
map_soho